<a href="https://colab.research.google.com/github/SproutCoder/text_mining_23/blob/main/project_2_lda_tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 2: Topic Modelling using BOW and Latent Dirichlet Allocation

For this project, imagine you are a group of data scientists who want to categorize news articles, without prior knowledge on their topics. Vectorize the documents using the *bag of words* method and apply *Latent Dirichlet Allocation* for topic modelling.


For this project, you might need the following python packages:
- sklearn
- pandas
- gensim
- string

### Enter names and mat. numbers:
- name 1, mat 1
- name 2, mat 2
- name 3, mat 3
- name 4, mat 4
- name 5, mat 5

## Task 1: Preprocessing

Perform the following preprocessing steps on the given dataset `data.csv`.

1. remove **duplicates**
2. **Tokenization**
3. Remove **stopwords**
4. Remove **punctuations**
5. **Lemmatization**: change words in third person to first person and verbs in past and future tenses into present.
6. **Stemming**: use a porter stemmer


### 1.1 Import Packages and Data

In [ ]:
import pandas as pd
import nltk #new line
nltk.download('punkt') #new line
nltk.download('stopwords') #new line
nltk.download('averaged_perceptron_tagger') #new line
nltk.download('wordnet') #new line
from nltk.corpus import stopwords, wordnet
from nltk.stem import porter, WordNetLemmatizer, PorterStemmer
from nltk import word_tokenize
import re
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# import data set
df = pd.read_csv('articles.csv', encoding='latin')

#remove duplicates
print('shape with duplicates: ', df.shape)
df.drop_duplicates(subset=['Article'], inplace=True)
print('shape having removed duplicates:', df.shape)
df.head()

shape with duplicates:  (2692, 4)
shape having removed duplicates: (2584, 4)


,Article,Date,Heading,NewsType
0,KARACHI: The Sindh government has decided to b...,1/1/2015,sindh govt decides to cut public transport far...,business
1,HONG KONG: Asian markets started 2015 on an up...,1/2/2015,asia stocks up in new year trad,business
2,HONG KONG: Hong Kong shares opened 0.66 perce...,1/5/2015,hong kong stocks open 0.66 percent lower,business
3,HONG KONG: Asian markets tumbled Tuesday follo...,1/6/2015,asian stocks sink euro near nine year,business
4,NEW YORK: US oil prices Monday slipped below $...,1/6/2015,us oil prices slip below 50 a barr,business


### 1.2 Define Preprocessing Function

In [ ]:
def preprocess(document: str):
  # tokenize
  document = re.sub(r'\W+', ' ', document)  # Replace non-alphanumeric characters with spaces so we also catch strings like "reported.Sources"
  tokens = word_tokenize(document)  # Tokenize the resulting string

  # remove stop words
  stop_words = set(stopwords.words('english')) # create a set of English stopwords
  filtered_tokens = [word for word in tokens if word.lower() not in stop_words] # remove stopwords

  # remove punctuation
  clean_tokens = [token for token in tokens if token not in string.punctuation]
  
  
  # perform POS tagging (needed for lemmatization)
  tagged_tokens = nltk.pos_tag(tokens) # => e.g. ('dogs', 'NNS'); uses Penn Treebank POS, which are not compatible with WordNet

  #convert Penn Treebank POS tags to WordNet POS tags 
  def convert_to_wordnet_pos(penn_pos):
      if penn_pos.startswith('J'):
          return wordnet.ADJ
      elif penn_pos.startswith('V'):
          return wordnet.VERB
      elif penn_pos.startswith('N'):
          return wordnet.NOUN
      elif penn_pos.startswith('R'):
          return wordnet.ADV
      else:
          return wordnet.NOUN  # noun as default

  # lemmatize each token based on its WordNet POS tag
  lemmatizer = WordNetLemmatizer()
  lemmatized_tokens = list(map(lambda x: lemmatizer.lemmatize(x[0], convert_to_wordnet_pos(x[1])), tagged_tokens))

  # stem tokens
  stemmer = PorterStemmer()
  stemmed_tokens = [stemmer.stem(token) for token in lemmatized_tokens]
  return stemmed_tokens

### 1.3 Apply Preprocessing Function to Data

In [ ]:
pre_df = pd.DataFrame()
pre_df['Article'] = df['Article'].apply(preprocess)
pre_df['Heading'] = df['Heading'].apply(preprocess)
pre_df['Date'] = df['Date']
pre_df['NewsType'] = df['NewsType']
pre_df.head()

,Article,Heading,Date,NewsType
0,"[karachi, the, sindh, govern, have, decid, to,...","[sindh, govt, decid, to, cut, public, transpor...",1/1/2015,business
1,"[hong, kong, asian, market, start, 2015, on, a...","[asia, stock, up, in, new, year, trad]",1/2/2015,business
2,"[hong, kong, hong, kong, share, open, 0, 66, p...","[hong, kong, stock, open, 0, 66, percent, low]",1/5/2015,business
3,"[hong, kong, asian, market, tumbl, tuesday, fo...","[asian, stock, sink, euro, near, nine, year]",1/6/2015,business
4,"[new, york, us, oil, price, monday, slip, belo...","[u, oil, price, slip, below, 50, a, barr]",1/6/2015,business


## Task 2: Latent Dirichlet Allocation

Apply LDA to the tokenized dataset using `gensim`.

1. Build a vocabulary using the gensim `Dictionary`
2. filter outliers: words that occur in less than 20 documents of in more than 50% of the documents.
3. create a BOW matrix for the preprocessed data
4. Train the LDA model on the training data
5. create a dataframe in which you map each document to a) its topic b) the topic's keywords

In [ ]:
# your code here

## Task 3: Discussion

Try different parameters for the LDA Model. What kind effects do you expect / observe?